In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load data from clipboard (assuming tab-separated values)
data = pd.read_clipboard(sep='\t')

# Splitting the data into train and test
np.random.seed(1234)
train, test = train_test_split(data, test_size=0.2, random_state=1234)

# Separating independent variables (IVs) and dependent variable (DV)
train_target = train.iloc[:, 13]  # Assuming target variable is in column index 13 (Python is zero-indexed)
test_target = test.iloc[:, 13]

train_features = train.iloc[:, :13]  # First 13 columns as features
test_features = test.iloc[:, :13]

# Convert features to NumPy matrix
train_features = train_features.to_numpy()
test_features = test_features.to_numpy()

# Define parameter grid for tuning
param_grid = {
    'n_estimators': list(range(200, 1100, 100)),
    'learning_rate': [0.05, 0.10, 0.15],
    'max_depth': [2, 3, 4, 5],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 2, 3]
}

# Initialize the XGBoost classifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=1234)

# Perform grid search with cross-validation
cv = GridSearchCV(xgb, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
cv.fit(train_features, train_target)

# Best parameters and model evaluation
best_model = cv.best_estimator_
predictions = best_model.predict(test_features)
accuracy = accuracy_score(test_target, predictions)

# Display results
print("Best Hyperparameters:", cv.best_params_)
print("Test Accuracy:", accuracy)
